<a href="https://colab.research.google.com/github/gulwish/ColabTasks/blob/main/Quantitative%20prediction%20of%20ted%20talks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting TED Talks Views with ML Models
In this notebook I've done some simple feature engineering on the TED Talks dataset and I've built machine learning models (Random Forest, XGBRegressor, ExtraTreesRegressor, and LGBMRegressor) and optimised their hyperparameters to predict the number of TED Talks views. I've made use of the following kernels to create this notebook:
* https://www.kaggle.com/rounakbanik/ted-data-analysis
* https://www.kaggle.com/holfyuen/what-makes-a-popular-ted-talk
* https://www.kaggle.com/tristanmoser/predicting-a-powerful-idea-a-ted-talk-analysis



In [2]:
# input data files are available in the "../input/" directory
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
import os 
os.environ['KAGGLE_CONFIG_DIR']='/content'

# load libraries
import time
import warnings
import random
import pandas as pd
import datetime
import lightgbm as lgb
import xgboost as xgb
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics
from mlxtend.regressor import StackingCVRegressor

# ignore warnings
warnings.filterwarnings('ignore')

# load data
data = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/TEDTalkViewsPrediction/ted_main.csv")
data.shape

FileNotFoundError: ignored

In [ ]:
data.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# TED Talks Data Analysis
## Cleaning up the data
Various datasets frequently have missing values, so I start off by checking whether the TED Talks dataset has any. 

In [ ]:
pd.isnull(data).sum()

There are only 6 null values in the **speaker_occupation** feature, I will fill in those missing values with a default 'Other' value.

In [ ]:
for index, row in data.iterrows():
    if pd.isnull(row['speaker_occupation']):
        data['speaker_occupation'][index] = 'Other'

## related_talks
Here I print out the **related_talks** feature so I check what it looks like.

In [ ]:
data['related_talks'][0]

After that I split the string by its commas and then by the semi-column to get the middle element, which is the views of all related talks.

In [ ]:
data['related_views'] = 0

for index, row in data.iterrows():
    vids = row['related_talks'].split(',')
    counter = 0
    total = 0
    for views in vids:
        if 'viewed_count' in views:
            view = views.split(':')
            # get rid of brackets and spaces
            view[1] = view[1].replace("]", "")
            view[1] = view[1].replace(" ", "")
            view[1] = view[1].replace("}", "")
            total+=int(view[1])
            counter+=1
    data['related_views'][index] = total/counter

## published_date, filmed_date
From these two features I extract day of the week, month, and year.

In [ ]:
data['published_date'] = data['published_date'].apply(lambda x: datetime.date.fromtimestamp(int(x)))
data['day'] = data['published_date'].apply(lambda x: x.weekday())
data['month'] = data['published_date'].apply(lambda x: x.month)
data['year'] = data['published_date'].apply(lambda x: x.year)
data['film_date'] = data['film_date'].apply(lambda x: datetime.date.fromtimestamp(int(x)))
data['day_film'] = data['film_date'].apply(lambda x: x.weekday())
data['month_film'] = data['film_date'].apply(lambda x: x.month)
data['year_film'] = data['film_date'].apply(lambda x: x.year)

Here I categorise the data which is preferable over using numbers.

In [ ]:
to_cat = {"day":   {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thurday", 4: "Friday", 5: "Saturday",
                    6: "Sunday" },
          "month": {1: "January", 2: "February", 3: "March", 4: "April", 5: "May", 6: "June", 7: "July", 8: "August", 
                    9: "September", 10: "October", 11: "November", 12: "December"},
          "year":  {2006: "2006", 2007: "2007", 2008: "2008", 2009: "2009", 2010: "2010", 2011: "2011", 2012: "2012", 
                    2013: "2013", 2014: "2014", 2015: "2015", 2016: "2016", 2017: "2017"},
          "day_film":   {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thurday", 4: "Friday", 5: "Saturday",
                    6: "Sunday" },
          "month_film": {1: "January", 2: "February", 3: "March", 4: "April", 5: "May", 6: "June", 7: "July", 8: "August", 
                    9: "September", 10: "October", 11: "November", 12: "December"},
          "year_film":  {2006: "2006", 2007: "2007", 2008: "2008", 2009: "2009", 2010: "2010", 2011: "2011", 2012: "2012", 
                    2013: "2013", 2014: "2014", 2015: "2015", 2016: "2016", 2017: "2017"}}

data.replace(to_cat, inplace=True)

## event
I check the number of unique event names then list all of them:

In [ ]:
print('Number of unique events: ',data['event'].unique().shape[0])
data['event'].unique()

The dataset has 355 unique event names but from the looks of it, lots of these names can be categorised together as they are quite similar. I break down the event names in the following 11 categories (each consisting of at least 5 samples).

In [ ]:
# initialise all values as 'Other' to assign this category
# to all entries that don't fit into the chosen categories
data['event_category'] = 'Other'

for i in range(len(data)):
    if data['event'][i][0:5]=='TED20':
        data['event_category'][i] = 'TED2000s'
    elif data['event'][i][0:5]=='TED19':
        data['event_category'][i] = 'TED1900s'
    elif data['event'][i][0:4]=='TEDx':
        data['event_category'][i] = "TEDx"
    elif data['event'][i][0:7]=='TED@BCG':
        data['event_category'][i] = 'TED@BCG'
    elif data['event'][i][0:4]=='TED@':
        data['event_category'][i] = "TED@"
    elif data['event'][i][0:8]=='TEDSalon':
        data['event_category'][i] = "TEDSalon"
    elif data['event'][i][0:9]=='TEDGlobal':
        data['event_category'][i] = 'TEDGlobal'
    elif data['event'][i][0:8]=='TEDWomen':
        data['event_category'][i] = 'TEDWomen'
    elif data['event'][i][0:6]=='TEDMED':
        data['event_category'][i] = 'TEDMED'
    elif data['event'][i][0:3]=='TED':
        data['event_category'][i] = 'TEDOther'

I check whether each categoies can be found in the dataset.

In [ ]:
data['event_category'].unique()

In [ ]:
data.head(1)


## tags
Using KeyedVectors to encode and categorise the information from the provided tags.

## Final touches on the dataset
I take out the views and comments and the rest of the features I won't be using.

In [ ]:
views = data['views']
comments = data['comments']
data = data.drop(['comments', 'description', 'event', 'film_date', 'main_speaker', 'name', 'published_date', 'ratings', 
           'related_talks', 'tags', 'title', 'url', 'views', 'speaker_occupation', 'Tags'], 1)
data.head()

In [ ]:
#data is datframe
#csv_data = data.to_csv() 


data.to_csv(r'C:\Users\mahad\Documents\cleaned_dataframe.csv', index=False, header=True)



In [ ]:
print(data.columns.tolist())

I apply **One-Hot-Encoding** on the categorical attributes and get the data ready for training machine learning models. Then I print out the dimensions of the final dataset.

In [ ]:
# data2 = data.filter(like='_tag', axis=1)
# data = data.drop(data2.columns, 1)
# data.head()
data_final = pd.get_dummies(data)
data_final.shape

# Machine Learning
## Random Forest
First model I will be testing with is **Random Forest** as this is the one I'm most comfortable with. I will then explore some other ML models, optimise the hyperparameters of each model, and combine those into an ensebmle model.

I split the dataset in training (90%) and test (10%) sets. The test data will be later used to validate the ML models on unseen data. I start off with a **Random Forest** as it's quite a powerful model that can be used as a baseline. I will use **Mean Absolute Error (MAE)** to measure the error as it will give us a more intuitive understanding of how accurate the model is. Additionally, using **Mean Squared Error (MSE)** to predict target variables with large values (such as the TED Talks views I'm working with) can lead to problems.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_final, views, test_size=0.1, random_state=121212)

The baseline Random Forest model seems to have overfitted. Together with the **MAE** of the model I also print the **mean** and **std** of the target variables. Judging by the high variance of the data, it's safe to conclude that the model is performing reasonably well.

In [ ]:
rf = RandomForestRegressor(criterion='mae',max_depth=15, max_features=45, n_estimators=500, min_samples_leaf=2, min_samples_split=2,
                           random_state=2019)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)
print('Training MAE: {:0.2f}'.format(metrics.mean_absolute_error(y_train, y_pred)))
print('Test MAE: {:0.2f}'.format(metrics.mean_absolute_error(y_test, y_test_pred)))
print('Views mean: {:0.2f}'.format(views.mean()))
print('Views std: {:0.2f}'.format(views.std()))

I plot the feature importances that are derived from the model that was just trained. I will use this in the future to exclude the unimportant features with the hope of boosting performance.

In [ ]:
importances = pd.DataFrame({'Features': X_train.columns, 
                                'Importances': rf.feature_importances_})
    
importances.sort_values(by=['Importances'], axis='index', ascending=False, inplace=True)
fig = plt.figure(figsize=(14, 4))
sns.barplot(x='Features', y='Importances', data=importances)
plt.xticks(rotation='vertical')
plt.show()

## XGBRegressor
Good accuracy, the model tends to overfit quite easily when **n_estimators > 20**. Maybe it's worth exploring wether we can use a higher **n_estimators** value while using the other hyperparameters to reguralise the model.

In [ ]:
xgbr = xgb.XGBRegressor(criterion='mae', earning_rate=0.1, max_depth=10, subsample=0.5, n_estimators=20, min_child_weight=2, random_state=2019)
xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_train)
y_test_pred = xgbr.predict(X_test)
print('Training MAE: {:0.2f}'.format(metrics.mean_absolute_error(y_train, y_pred)))
print('Test MAE: {:0.2f}'.format(metrics.mean_absolute_error(y_test, y_test_pred)))

## ExtraTreesRegressor
ExtraTreesRegressor yields the best accuracy, maybe can reguralise the model better as the gap between Training and Test MAE is quite big?

In [ ]:
et = ExtraTreesRegressor(criterion='mae', max_depth=30, n_estimators=1000, random_state=2019, min_samples_leaf=2, min_samples_split=6)
et.fit(X_train, y_train)
y_pred = et.predict(X_train)
y_test_pred = et.predict(X_test)
print('Training MAE: {:0.2f}'.format(metrics.mean_absolute_error(y_train, y_pred)))
print('Test MAE: {:0.2f}'.format(metrics.mean_absolute_error(y_test, y_test_pred)))